# Group codes

In [1]:
# common imports
%matplotlib inline
import numpy as np
import pandas as pd
from tabulate import tabulate
from pymongo import MongoClient
import matplotlib.pyplot as plt
plt.style.use('seaborn')
plt.rcParams["figure.figsize"] = (20,8)

db = MongoClient()['stores']

TOTAL_NUMBER_OF_RECORDS = db.data.count()
NUMBER_OF_UNIQ_CODES = len(db.data.distinct("code"))

results = db.data.aggregate(
    [
        {
            "$group": {
                "_id": "$code",
                "distributors": {"$addToSet": "$source"},
                "count": {"$sum": 1},
            },
        },
        {
            "$project": {
                "_id": 1,
                "distributors_number": {"$size": "$distributors"},
                "distributors": 1,
                "count": 1,
            },
        },
        {
            "$sort": {
                "count": -1,
            },
        },
    ]
)

CODES_WITH_DISTRIBUTORS = [
    (x['_id'], 
     len(x['_id']),
     x['count'], 
     x['distributors_number'],
     ", ".join([i[:3] for i in x['distributors']]))
    for x in list(results)
]

print('Records: {} Uniq. codes: {}'.format(
    TOTAL_NUMBER_OF_RECORDS, NUMBER_OF_UNIQ_CODES))
codes_length = [x[1] for x in CODES_WITH_DISTRIBUTORS]
print('Min code length: {} Max code length: {}'.format(
    min(codes_length), max(codes_length)))

table = [(x[0], x[1], x[2], x[3], x[4]) for x in CODES_WITH_DISTRIBUTORS[:50]]
print(tabulate(table, 
               headers=['Code', 'Len', 'Count', '# of Distr.', 'Distrs'], 
               tablefmt="simple"))

Records: 1561159 Uniq. codes: 20078
Min code length: 1 Max code length: 69
Code      Len    Count    # of Distr.  Distrs
------  -----  -------  -------------  ---------------------------------------------------------------
874         3     3219              4  mar, den, evi, ign
5180        4     3098             12  rub, kin, evi, pat, maj, mar, all, tit, ros, ign, lar, den
2000        4     3060              7  yol, lar, kin, tit, ign, all, mar
5250        4     2910              9  joa, lar, tit, kin, maj, all, mar, rub, pat
5000        4     2801              6  yol, kin, lar, tit, all, mar
29M         3     2764              8  joa, kin, all, ros, ign, pat, evi, den
980         3     2521             10  joa, kin, evi, mar, all, ros, ign, pat, lar, den
5170        4     2392             12  rub, kin, evi, pat, maj, mar, all, tit, ros, ign, lar, den
4400        4     2329              9  evi, mar, all, tit, ros, ign, pat, lar, den
3001        4     2328              6  yol, maj, 

There is extremes like code length 1 or code length 69. Let's create new collection with calculated field `code_len`

In [2]:
db.data.aggregate(
    [
        {
            "$group": {
                "_id": "$code",
                "distributors": {"$addToSet": "$source"},
                "count": {"$sum": 1},
            }
        },
        {
            "$project": {
                "_id": 1,
                "num_of_distrs": {"$size": "$distributors"},
                "distributors": 1,
                "products_count": 1,
            },
        },
        {
            "$out": "code_mapping"
        }
        
    ]
)
print('Db "code_mapping" created')

# calculate code length 
# (special commands available only since mongodb 3.4)
print("Calculating code length")
uniq_codes = db.code_mapping.distinct("_id")
for code in uniq_codes:
    db.code_mapping.update_many(
        {"_id": code},
        {"$set": {"code_len": len(code), "code": ""}}
    )
    # print("Length for code: {} - {}".format(code, len(code)))
print("Done.")

# create indexes
print("Indexing...")
db.code_mapping.create_index([("code_len", 1)])
db.code_mapping.create_index([("products_count", 1)])
db.code_mapping.create_index([("num_of_distrs", 1)])
print('Done.')

print(list(db.code_mapping.find().limit(5)))

Db "code_mapping" created
Calculating code length
Done.
Indexing...
Done.
[{'code': '', 'code_len': 4, '_id': 'X864', 'distributors': ['rubi'], 'num_of_distrs': 1}, {'code': '', 'code_len': 4, '_id': 'X710', 'distributors': ['rubi'], 'num_of_distrs': 1}, {'code': '', 'code_len': 4, '_id': 'X70J', 'distributors': ['rubi'], 'num_of_distrs': 1}, {'code': '', 'code_len': 4, '_id': 'X372', 'distributors': ['rubi'], 'num_of_distrs': 1}, {'code': '', 'code_len': 4, '_id': 'X245', 'distributors': ['rubi'], 'num_of_distrs': 1}]


In [67]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


def smart_code_extractor(x):
    """Pre-processor"""
    if '(' not in x and ')' not in x:
        if len(x) > 8:
            tokens = x.split(' ')
            return tokens[-1].strip()
        else:
            if 'GILDAN' in x:
                return 'G' + x.split(' ')[1].strip()
            elif 'TRIBLEND' in x:
                return 'T' + x.split(' ')[1].strip()
        
    if '(' in x and ')' in x:
        token = x.split('(')[0]
        return token.strip()
    
    return x.strip()
    
def code_preprocessor(x):
    """Split code into the chars delimited by ` ` (space)"""
    token = smart_code_extractor(x).replace(
        ' ', '').replace(
        '/', '').replace(
        ',', '').replace(
        '-', '').replace(
        '(', '')
    return "".join(list(token.strip()))
    
CODES_LIST = [x[0] for x in CODES_WITH_DISTRIBUTORS]
count_vectorizer = CountVectorizer(
    preprocessor=code_preprocessor, 
    analyzer='char', 
    lowercase=False, 
    ngram_range=(1, 2),
    min_df=5, max_df=0.95,
)
count_vectorizer = TfidfVectorizer(
    
)
print(count_vectorizer)


CountVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=0.95, max_features=None, min_df=5,
        ngram_range=(1, 2),
        preprocessor=<function code_preprocessor at 0x107460510>,
        stop_words=None, strip_accents=None,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, vocabulary=None)


In [68]:
cv = count_vectorizer.fit_transform(CODES_LIST)
cv_feature_names = count_vectorizer.get_feature_names()
print(cv_feature_names[:10])
print(CODES_LIST[:10])
print(cv.toarray()[:10])
print(repr(cv))

['0', '00', '01', '02', '03', '04', '05', '06', '07', '08']
['874', '5180', '2000', '5250', '5000', '29M', '980', '5170', '4400', '3001']
[[0 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 [3 2 0 ..., 0 0 0]
 ..., 
 [1 0 0 ..., 0 0 0]
 [2 1 0 ..., 0 0 0]
 [2 1 1 ..., 0 0 0]]
<20078x876 sparse matrix of type '<class 'numpy.int64'>'
	with 173385 stored elements in Compressed Sparse Row format>


In [63]:
# pick G200 and G2000 indexes and calculate cosine similarity
v1_index = CODES_LIST.index('G200')
v2_index = CODES_LIST.index('G2000')
test_index = CODES_LIST.index('G500')
print(v1_index, v2_index)
v1 = cv[v1_index]
v2 = cv[v2_index]
vt = cv[test_index]
print(v1.toarray())
print(v2.toarray())
print(vt)

from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(v1, v2)
print("Similarity G200 vs G2000 : ", similarity)
similarity = cosine_similarity(v1, vt)
print("Similarity G200 vs G500 : ", similarity)
similarity = cosine_similarity(v2, vt)
print("Similarity G2000 vs G500 : ", similarity)

31 30
[[2 1 0 ..., 0 0 0]]
[[3 2 1 ..., 0 0 0]]
  (0, 2535)	1
  (0, 2534)	1
  (0, 2489)	1
  (0, 1103)	1
  (0, 1102)	1
  (0, 1)	1
  (0, 0)	2
  (0, 1101)	1
Similarity G200 vs G2000 :  [[ 0.94387981]]
Similarity G200 vs G500 :  [[ 0.54545455]]
Similarity G2000 vs G500 :  [[ 0.60677988]]


In [75]:
# Calculate similarity of G200

v1_index = CODES_LIST.index('G200')
v1 = cv[v1_index]
results = []
for c, v in zip(CODES_LIST, cv):
    norm_vector = np.array([[2., 1.2] + (874 * [1.])])
#     print(norm_vector.shape)
#     print(v1.shape)
    similarity = cosine_similarity(v1, v)
    results.append((c, similarity))

(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(

(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(

(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(

(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(

(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(

...

(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(

(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(

(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(

(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(

(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)
(1, 876)


In [77]:
# sort with CODES LIST
sorted_results = sorted(results, key=lambda x: x[1], reverse=True)
table = []
for code, cosine in sorted_results[:50]:
    table.append((code, cosine[0][0]))
    
print(tabulate(table, 
               headers=['Code', 'Similarity'], 
               tablefmt="simple"))

Code              Similarity
--------------  ------------
02000B/02000BX      0.341518
01000/01000X        0.311789
02006/02006X        0.287542
01005/010052X       0.254678
0100B/01001BX       0.243427
03003/03003X        0.242468
G2000               0.232824
G2000B              0.23207
G2000L              0.23207
G2000T              0.23207
G2000P              0.23207
AG2000              0.23207
02000               0.23182
G42000              0.221123
G12000              0.221123
G92000              0.221123
G42000L             0.220411
G42000B             0.220411
US2000G             0.220411
00006               0.220364
00001               0.220364
US2000GD            0.219706
US2000GS            0.219472
01102/01102X        0.214603
G2200               0.211596
2000                0.21155
GILDAN 2000         0.21155
BELLA 2000          0.21155
AG2200              0.210905
2000L               0.21086
2000B               0.21086
42000               0.21086
12000               0.2108

In [23]:
cv = count_vectorizer.fit_transform(CODES_LIST)
cv_feature_names = count_vectorizer.get_feature_names()
print(cv_feature_names)
print()
print(cv.toarray())
print()
print(repr(cv))

['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '0A', '0B', '0C', '0D', '0E', '0F', '0G', '0H', '0I', '0J', '0L', '0M', '0N', '0O', '0P', '0R', '0S', '0T', '0U', '0V', '0W', '0X', '0Y', '0Z', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '1A', '1B', '1C', '1D', '1E', '1G', '1I', '1J', '1L', '1M', '1N', '1O', '1P', '1R', '1S', '1T', '1U', '1V', '1W', '1Y', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '2A', '2B', '2C', '2D', '2E', '2G', '2H', '2I', '2J', '2L', '2M', '2N', '2O', '2P', '2R', '2S', '2T', '2U', '2V', '2W', '2X', '2Y', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '3A', '3B', '3C', '3D', '3E', '3G', '3H', '3I', '3J', '3L', '3M', '3N', '3O', '3P', '3R', '3S', '3T', '3V', '3W', '3X', '3Y', '3Z', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '4A', '4B', '4C', '4D', '4E', '4F', '4G', '4H', '4I', '4J', '4L', '4M', '4N', '4O', '4P', '4R', '4S', '4T', '4V', '4W', '4Y', '50', '51', '52', '53', '54', '55', '56', '57

In [25]:
print(count_vectorizer.vocabulary_)

{'FF': 452, '5I': 176, 'K3': 544, 'R0': 681, '1M': 53, 'NF': 622, 'RO': 698, '3A': 106, 'K2': 543, '45': 133, 'WT': 806, 'DL': 414, 'OR': 643, 'C3': 368, 'V1': 775, 'CD': 378, 'XP': 818, 'H6': 489, 'FP': 456, 'FS': 458, 'EB': 432, 'MI': 601, 'T4': 738, '68': 199, 'N4': 613, '33': 99, '4F': 143, '3L': 115, 'TG': 749, 'V3': 777, 'D1': 397, 'FX': 461, '7Y': 246, 'BB': 344, '6L': 209, 'U0': 762, '42': 130, 'X5': 814, '9G': 292, 'L8': 571, '6W': 218, 'O4': 630, '56': 165, 'G3': 465, '1G': 49, 'S4': 709, 'MD': 600, 'X1': 810, 'P8': 653, '9S': 299, 'P4': 649, 'KR': 559, '06': 6, '6V': 217, '6O': 212, '0G': 16, 'AF': 319, 'RM': 697, '23': 67, 'F0': 442, 'F9': 449, 'G6': 468, 'SR': 728, 'T1': 735, '5V': 187, '73': 223, 'BO': 354, 'HF': 497, 'ST': 730, 'EX': 441, 'S6': 711, 'CE': 379, '5P': 182, '74': 224, 'OK': 640, '7O': 239, 'BZ': 364, '1Y': 63, 'BR': 356, 'O2': 628, 'YR': 831, 'RE': 693, 'DC': 408, '48': 136, 'J1': 526, 'AL': 323, 'DP': 417, '0S': 26, '49': 137, '9O': 296, 'FC': 451, 'CA': 3

In [89]:
# from sklearn.cluster import AffinityPropagation

# af = AffinityPropagation(preference=-50).fit(data.components_)
# cluster_centers_indices = af.cluster_centers_indices_
# labels = af.labels_

# n_clusters_ = len(cluster_centers_indices)

# print('Estimated number of clusters: %d' % n_clusters_)

Estimated number of clusters: 1
